In [1]:
# Dependencies
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import geocoder # import geocoder
import requests 
from bs4 import BeautifulSoup 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# map rendering library
import folium

# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize 

print('Libraries imported.')

Libraries imported.


In [2]:
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
reqs = requests.get(URL)

soup = BeautifulSoup(reqs.content, 'html5lib') 
table = soup.find('div', attrs = {'id':'container'}) 

#print(soup.prettify()) 
print('Done.')

Done.


In [3]:
#put relevant data in python lists

postalCodes = [];
boroughs= [];
neighbourhoods = [];
columnNum = 1;
passVal = False

for row in soup.find_all('td'):
    for cell in row:
        if cell.string and cell.string[0].isalpha() and len(cell.string) > 2:
            passVal = False
            if columnNum == 1:
                if passVal == False and cell.string[1].isdigit():
                    postalCodes.append(cell.string);   
                    columnNum = 2
                else:
                    continue
            elif columnNum == 2 :
                if cell.string == 'Not assigned':
                    passVal = True
                    del postalCodes[-1]
                    columnNum = 1
                    continue
                else:
                    boroughs.append(cell.string);      
                    columnNum = 3
            elif columnNum == 3 :
                if cell.string == 'Not assigned\n':
                    neighbourhoods.append(boroughs[-1])
                else:
                    neighbourhoods.append(cell.string); 
                columnNum = 1
                
print('Data Collected.')

Data Collected.


In [4]:
print(postalCodes)
print(boroughs)
print(neighbourhoods)

['M3A', 'M4A', 'M5A', 'M6A', 'M6A', 'M7A', 'M9A', 'M1B', 'M1B', 'M3B', 'M4B', 'M4B', 'M5B', 'M5B', 'M6B', 'M9B', 'M9B', 'M9B', 'M9B', 'M9B', 'M1C', 'M1C', 'M1C', 'M3C', 'M3C', 'M4C', 'M5C', 'M6C', 'M9C', 'M9C', 'M9C', 'M9C', 'M1E', 'M1E', 'M1E', 'M4E', 'M5E', 'M6E', 'M1G', 'M4G', 'M5G', 'M6G', 'M1H', 'M2H', 'M3H', 'M3H', 'M3H', 'M4H', 'M5H', 'M5H', 'M5H', 'M6H', 'M6H', 'M1J', 'M2J', 'M2J', 'M2J', 'M3J', 'M3J', 'M4J', 'M5J', 'M5J', 'M5J', 'M6J', 'M6J', 'M1K', 'M1K', 'M1K', 'M2K', 'M3K', 'M3K', 'M4K', 'M4K', 'M5K', 'M5K', 'M6K', 'M6K', 'M6K', 'M1L', 'M1L', 'M1L', 'M2L', 'M2L', 'M3L', 'M4L', 'M4L', 'M5L', 'M5L', 'M6L', 'M6L', 'M6L', 'M9L', 'M1M', 'M1M', 'M1M', 'M2M', 'M2M', 'M3M', 'M4M', 'M5M', 'M5M', 'M6M', 'M6M', 'M6M', 'M6M', 'M9M', 'M9M', 'M1N', 'M1N', 'M2N', 'M3N', 'M4N', 'M5N', 'M6N', 'M6N', 'M9N', 'M1P', 'M1P', 'M1P', 'M2P', 'M4P', 'M5P', 'M5P', 'M6P', 'M6P', 'M9P', 'M1R', 'M1R', 'M2R', 'M4R', 'M5R', 'M5R', 'M5R', 'M6R', 'M6R', 'M7R', 'M9R', 'M9R', 'M9R', 'M9R', 'M1S', 'M4S', 'M5S'

In [5]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
hoods = pd.DataFrame(columns=column_names)

hoods

,PostalCode,Borough,Neighbourhood,Latitude,Longitude


In [6]:
#helper code
# initialize your variable to None
lat_lng_coords = None

for data in range(0, len(postalCodes)-1):
    code = postalCodes[data]
    borough = boroughs[data]
    neighbourhood_name = neighbourhoods[data]
    
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    lat_lng_coords = g.latlng

    hoods = hoods.append({ 'PostalCode': code,
                                   'Borough': borough,
                                   'Neighbourhood': neighbourhood_name,
                                   'Latitude': lat_lng_coords[0],
                                   'Longitude': lat_lng_coords[1]}, ignore_index=True)

In [7]:
#check the data
print(hoods.head())
hoods.shape


  PostalCode           Borough     Neighbourhood   Latitude  Longitude
0        M3A        North York         Parkwoods  43.752420 -79.329242
1        M4A        North York  Victoria Village  43.730600 -79.313265
2        M5A  Downtown Toronto      Harbourfront  43.650295 -79.359166
3        M6A        North York  Lawrence Heights  43.723270 -79.451286
4        M6A        North York    Lawrence Manor  43.723270 -79.451286


(209, 5)

In [8]:
#Foursquare helper code
CLIENT_ID = 'DZV5VEPTZLAMZ2OHWJJIBOIIBWN3RSI3141U25BE0FNMESL5' # your Foursquare ID
CLIENT_SECRET = 'V5JMVQAUU13TGDNCPOMWRF2UE4KMT5TGT1M2VHUW4THR1QAL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DZV5VEPTZLAMZ2OHWJJIBOIIBWN3RSI3141U25BE0FNMESL5
CLIENT_SECRET:V5JMVQAUU13TGDNCPOMWRF2UE4KMT5TGT1M2VHUW4THR1QAL


In [9]:
#Foursuqare helper code
neighbourhood_latitude = hoods.loc[0, 'Latitude'] # neighbourhood latitude value
neighbourhood_longitude = hoods.loc[0, 'Longitude'] # neighbourhood longitude value

neighbourhood_name = hoods.loc[0, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Parkwoods are 43.75242000000003, -79.32924245299995.


In [10]:
#Forusquare helper code
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=DZV5VEPTZLAMZ2OHWJJIBOIIBWN3RSI3141U25BE0FNMESL5&client_secret=V5JMVQAUU13TGDNCPOMWRF2UE4KMT5TGT1M2VHUW4THR1QAL&v=20180605&ll=43.75242000000003,-79.32924245299995&radius=500&limit=100'

In [11]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e075ad7e826ac001bca8467'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.75692000450003,
    'lng': -79.32302427998279},
   'sw': {'lat': 43.74791999550003, 'lng': -79.33546062601711}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs

In [12]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [13]:
#clean and structure the JSON
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114


In [14]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


In [19]:
#function that returns nearby venues to a location
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [20]:
#Toronto venues
toronto_venues = getNearbyVenues(names=hoods['Neighbourhood'],
                                   latitudes=hoods['Latitude'],
                                   longitudes=hoods['Longitude']
                                  )

Parkwoods
Victoria Village
Harbourfront
Lawrence Heights
Lawrence Manor
Queen's Park
Queen's Park
Rouge
Malvern
Don Mills North

Woodbine Gardens
Parkview Hill
Ryerson

Garden District

Glencairn

Cloverdale

Islington
Martin Grove

Princess Gardens
West Deane Park
Highland Creek
Rouge Hill
Port Union
Flemingdon Park
Don Mills South

Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens

Eringate

Markland Wood
Old Burnhamthorpe

Guildwood

Morningside
West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks

Woburn
Leaside
Central Bay Street

Christie

Cedarbrae

Hillcrest Village
Bathurst Manor
Downsview North

Wilson Heights
Thorncliffe Park
Adelaide

King

Richmond

Dovercourt Village
Dufferin

Scarborough Village
Fairview

Henry Farm
Oriole

Northwood Park
York University
East Toronto
Harbourfront East

Toronto Islands
Union Station
Little Portugal
Trinity
East Birchmount Park

Ionview
Kennedy Park
Bayview Village
CFB Toronto
Downsview East

The Danforth West

Riverdal

In [21]:
# check the resulting dataframe

print(toronto_venues.shape)
toronto_venues.head()

(4714, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.752420,-79.329242,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.752420,-79.329242,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.730600,-79.313265,Wigmore Park,43.731023,-79.310771,Park
3,Victoria Village,43.730600,-79.313265,Memories of Africa,43.726602,-79.312427,Grocery Store
4,Harbourfront,43.650295,-79.359166,The Distillery Historic District,43.650244,-79.359323,Historic Site


In [22]:
# check how many venues were returned for each neighbourhood

toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Adelaide,100,100,100,100,100,100
Agincourt,14,14,14,14,14,14
Agincourt North,2,2,2,2,2,2
Albion Gardens,14,14,14,14,14,14
Alderwood,5,5,5,5,5,5
Bathurst Quay,68,68,68,68,68,68
Bayview Village,4,4,4,4,4,4
Beaumond Heights,14,14,14,14,14,14
Bedford Park,20,20,20,20,20,20


In [24]:
##Analyze Each Neighbourhood

In [25]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,...,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
#examine size
toronto_onehot.shape

(4714, 262)

In [27]:
#group rows by neighbourhood and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,...,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Adelaide,0.000000,0.00,0.030000,0.0,0.010000,0.000000,0.030000,0.000000,0.0,...,0.00,0.000000,0.0,0.010000,0.000000,0.000000,0.010000,0.000000,0.0,0.000000
1,Agincourt,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.00,0.000000,0.0,0.000000,0.000000,0.071429,0.000000,0.000000,0.0,0.000000
2,Agincourt North,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3,Albion Gardens,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
4,Alderwood,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.200000,0.0,...,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
5,Bathurst Quay,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.014706,0.000000,0.0,...,0.00,0.014706,0.0,0.000000,0.000000,0.000000,0.000000,0.014706,0.0,0.014706
6,Bayview Village,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.25,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
7,Beaumond Heights,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
8,Bedford Park,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
9,Berczy Park,0.000000,0.00,0.000000,0.0,0.015873,0.000000,0.000000,0.000000,0.0,...,0.00,0.000000,0.0,0.015873,0.000000,0.000000,0.000000,0.000000,0.0,0.000000


In [28]:
#see if size checks out
toronto_grouped.shape

(196, 262)

In [29]:
#print each neighbourhood along with the top 5 most common venues
#function that sorts the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [35]:
#display top10 venues for each neighbourhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Steakhouse,Hotel,Bar,Gastropub,Japanese Restaurant,Gym,Asian Restaurant,Restaurant
1,Agincourt,Chinese Restaurant,Shopping Mall,Hong Kong Restaurant,Badminton Court,Pool,Discount Store,Skating Rink,Supermarket,Sushi Restaurant,Bakery
2,Agincourt North,Pharmacy,Yoga Studio,Eastern European Restaurant,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
3,Albion Gardens,Grocery Store,Pizza Place,Coffee Shop,Fast Food Restaurant,Pharmacy,Liquor Store,Fried Chicken Joint,Caribbean Restaurant,Sandwich Place,Beer Store
4,Alderwood,Gym,Convenience Store,Sandwich Place,Athletics & Sports,Pub,Yoga Studio,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space


In [36]:
#clustering
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 4, 3, 0, 0, 3, 3, 0, 0])

In [37]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = hoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighbourhood
toronto_merged = hoods.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.752420,-79.329242,2.0,Food & Drink Shop,Park,Farmers Market,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Yoga Studio
1,M4A,North York,Victoria Village,43.730600,-79.313265,3.0,Grocery Store,Park,Yoga Studio,Farmers Market,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Fast Food Restaurant
2,M5A,Downtown Toronto,Harbourfront,43.650295,-79.359166,0.0,Coffee Shop,Bakery,Theater,Boat or Ferry,Ice Cream Shop,Park,Hotel,Performing Arts Venue,Pub,Gastropub
3,M6A,North York,Lawrence Heights,43.723270,-79.451286,0.0,Clothing Store,Pharmacy,Toy / Game Store,Coffee Shop,Men's Store,Food Court,Furniture / Home Store,Cosmetics Shop,American Restaurant,Leather Goods Store
4,M6A,North York,Lawrence Manor,43.723270,-79.451286,0.0,Clothing Store,Pharmacy,Toy / Game Store,Coffee Shop,Men's Store,Food Court,Furniture / Home Store,Cosmetics Shop,American Restaurant,Leather Goods Store


In [59]:
#clear NaNs!!!
toronto_merged['Cluster Labels'].isna().sum()
toronto_merged.dropna(inplace=True)

In [60]:
#visualize clusters
# create map
map_clusters = folium.Map(location=[43.75242000000003, -79.32924245299995], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]



In [61]:
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters